(optimization_latent notebook cells to be copied to original)

In [ ]:
import os

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder
from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

from params import lsh_test, hypercube_test, kmeans_test, gnn_test, mrng_test, nsg_test, get_aaf

In [ ]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'
    decoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_dataset.dat'
    decoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

# MRNG

In [ ]:
def objective_mrng(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'l': trial.suggest_int('l', 1, 1000)}
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]

    average_time, aaf_latent = mrng_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, N=5, int_data=0)

    return aaf_latent.value, average_time.value

In [ ]:
%%time
mrng_study = optuna.create_study(study_name='mrng', directions=['minimize', 'minimize'])
mrng_study.optimize(objective_mrng, n_trials=50, n_jobs=3)
print("-------------------- Best trials --------------------")
trials = sorted(mrng_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

In [ ]:
df = mrng_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

In [ ]:
plot_pareto_front(mrng_study, target_names=['aaf', 'average_time'])

In [ ]:
plot_optimization_history(mrng_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_optimization_history(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(mrng_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_slice(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')